In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
def qnorm_dataframe( data ):
    """
    quantile normalize a dataframe with numeric values only!
    """
    rank_mean = data.stack().groupby(data.rank(method='first').stack().astype(int)).mean()
    qnormed_data    = data.rank(method='min').stack().astype(int).map(rank_mean).unstack()
    return qnormed_data

In [3]:
def ut_as_list( dframe, diag=1, cols=['Row','Column','Value'] ):
  """
  for a symmetric dataframe, where cols=rows, get the upper triangle as a list of row/column pairs
  diag = 1 (default): ignore diagonal
  diag = 0: include diagonal
  """
  #if (dframe.index.name == dframe.columns.name):
  #dframe.index.name = cols[0]
  #dframe.columns.name = cols[1]
  #		dframe.index.name = dframe.index.name + '.1'
  #		dframe.index.name = dframe.index.name + '.2'
  d = dframe.where( np.triu( np.ones( dframe.shape ), k=diag).astype(np.bool))
  d.index.rename(cols[0], inplace=True)
  d.columns.rename(cols[1], inplace=True)
  d = d.stack().reset_index()
  d.columns=cols
  return d

In [4]:
bf = pd.read_table('./Data/table_Avana2020Q4_CRISPRcleanR_corrected_all', index_col=0)
qbf = qnorm_dataframe( bf )
qbf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18095 entries, A1BG to ZZZ3
Columns: 808 entries, ACH-001172 to ACH-002510
dtypes: float64(808)
memory usage: 111.7+ MB


In [6]:
%%time
cc = pd.DataFrame( index=qbf.index.values, columns=qbf.index.values, data=np.corrcoef(qbf))
cc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18095 entries, A1BG to ZZZ3
Columns: 18095 entries, A1BG to ZZZ3
dtypes: float64(18095)
memory usage: 2.4+ GB
CPU times: user 16.7 s, sys: 2.58 s, total: 19.3 s
Wall time: 2.45 s


In [5]:
#cc.to_csv('./Data/cc_all-matrix-qbf-18k.txt', sep=',', index=True, float_format='%4.3f')
cc = pd.read_table('./Data/cc_all-matrix-qbf-18k.txt', sep=',', index_col=0)

In [11]:
cc_list = ut_as_list( cc, diag=1, cols=['Gene1','Gene2','PCC'])

<ipython-input-3-5a1515b0419d>:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  d = dframe.where( np.triu( np.ones( dframe.shape ), k=diag).astype(np.bool))


In [12]:
cc_list.sort_values('PCC', ascending=False, inplace=True)

In [13]:
cc_list.to_csv('./Data/cc_all-list-qbf-18k.txt', sep='\t', index=False, float_format='%4.3f')